In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import ast
from IPython.display import display, HTML
import pyspark

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:51828 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 34.29 GB


In [24]:
# save test versions
pd.read_csv('matches_small.csv', nrows=1000).to_csv('matches_small_test.csv', index=False)
pd.read_csv('player_matches_small.csv', nrows=1000).to_csv('player_matches_small_test.csv', index=False)
pd.read_csv('match_skill.csv', nrows=1000).to_csv('match_skill_test.csv', index=False)


In [5]:
df_matches_small = dd.read_csv('matches_small.csv')
df_match_skill = dd.read_csv('match_skill.csv')
df_player_matches_small = dd.read_csv('player_matches_small.csv')

In [34]:
dd.read_csv('player_matches_small.csv').to_parquet('player_matches_small.parquet')

ValueError: ('Long error message', 'Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.\n\n+------------------+---------+----------+\n| Column           | Found   | Expected |\n+------------------+---------+----------+\n| ability_uses     | object  | float64  |\n| actions          | object  | float64  |\n| buyback_log      | object  | float64  |\n| damage           | object  | float64  |\n| damage_inflictor | object  | float64  |\n| damage_taken     | object  | float64  |\n| gold_reasons     | object  | float64  |\n| gold_t           | object  | float64  |\n| hero_hits        | object  | float64  |\n| item_uses        | object  | float64  |\n| kill_streaks     | object  | float64  |\n| killed           | object  | float64  |\n| killed_by        | object  | float64  |\n| kills_log        | object  | float64  |\n| lane_pos         | object  | float64  |\n| leaver_status    | float64 | int64    |\n| lh_t             | object  | float64  |\n| life_state       | object  | float64  |\n

In [20]:
ts = pd.DataFrame(df_matches_small['start_time'].head())
ts['date'] = pd.to_datetime(ts['start_time'], unit='s')
ts

,start_time,date
0,1461013929,2016-04-18 21:12:09
1,1461013382,2016-04-18 21:03:02
2,1461012679,2016-04-18 20:51:19
3,1461013865,2016-04-18 21:11:05
4,1461012987,2016-04-18 20:56:27


In [29]:
df_matches_small.columns

Index(['match_id', 'match_seq_num', 'radiant_win', 'start_time', 'duration',
       'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant',
       'barracks_status_dire', 'cluster', 'first_blood_time', 'lobby_type',
       'human_players', 'leagueid', 'positive_votes', 'negative_votes',
       'game_mode', 'engine', 'picks_bans', 'parse_status', 'chat',
       'objectives', 'radiant_gold_adv', 'radiant_xp_adv', 'teamfights',
       'version', 'pgroup'],
      dtype='object')

In [31]:
df_matches_small.cluster.value_counts().compute()

151    123224
153    122918
154    119986
152    112255
204     95907
132     85414
181     83581
224     80927
156     75890
227     70362
155     69356
133     69209
182     58426
231     54721
187     52854
121     50478
122     46402
191     46379
223     45900
188     43685
138     43077
185     37777
184     34321
192     33504
225     33379
123     32752
131     22380
183     22282
137     22021
186     21409
111     19569
112     19066
134     18167
136     17782
135     16508
171     14559
124     11941
144      6544
251      6346
145      5304
161      4351
213      3665
242      1474
241      1160
261      1053
211       861
212       179
200        78
113        61
114        52
203         6
143         5
202         4
141         2
142         2
Name: cluster, dtype: int64

In [ ]:
# transform to add columns for players 0-9: account_id and hero_id
def add_feature_columns(x):
    accounts = np.empty(10)
    accounts[:] = np.nan
    col_names = ['p0','p1','p2','p3','p4','p5','p6','p7','p8','p9']
    try:
        d = ast.literal_eval(x)
    except Exception as e:
        # invalid input
        return pd.Series(index = col_names, data=accounts)
    if isinstance(d, dict):
        # parse
        for i, (key, val) in enumerate(d.items()):
            try:
                accounts[i] = int(val['account_id'])
            except KeyError as e:
                pass
    return pd.Series(index = col_names, data=accounts)

In [ ]:
# may need to reread csv if this operation failed once
for col in ['p0','p1','p2','p3','p4','p5','p6','p7','p8','p9']:
    if col in df_matches_small.columns:
        df_matches_small = df_matches_small.drop(col, axis=1)
#df_matches_small = df_matches_small.join(df_matches_small['pgroup'].apply(add_feature_columns, 'axis=1'))
df_players = df_matches_small['pgroup'].apply(add_feature_columns, 'axis=1')
df_players['match_id'] = df_matches_small['match_id']
df_players
#client.compute(df_matches_small)

In [ ]:
%%time
dd.to_parquet(df_players, 'df_players_pk', engine='pyarrow')

In [ ]:
%%time
drop_cols = ['pgroup', 'cluster', 'negative_votes', 'positive_votes', 'parse_status']
df_matches_small = df_matches_small.drop(drop_cols, axis=1)
dd.to_parquet(df_matches_small, 'df_matches_small_pk.parquet', engine='pyarrow')

In [ ]:
pd.read_parquet('df_players_pk').head()

In [ ]:
print('test')

In [ ]:
df_match_skill.head()

In [ ]:
c_types = df_player_matches_small.dtypes
c_types = c_types.apply(str)
c_types = c_types.to_dict()
string_cols = ['max_hero_hit',
              'times',
               'gold_t',
               'lh_t',
               'xp_t',
               'obs_log',
               'sen_log',
               'purchase_log',
               'kills_log',
               'buyback_log',
               'lane_pos',
               'obs',
               'sen',
               'actions',
               'pings',
               'purchase',
               'gold_reasons',
               'xp_reasons',
               'killed',
               'item_uses',
               'ability_uses',
               'hero_hits',
               'damage_inflictor',
               'runes',
               'killed_by',
               'kill_streaks',
               'multi_kills',
               'life_state',
               'damage',
               'damage_taken',
               'additional_units']
# convert json fields to string
for c in string_cols:
    c_types[c] = 'str'
# convert int fields to float to handle NaNs
for key, val in c_types.items():
    if val == 'int64':
        c_types[key] = 'float64'
c_types

In [ ]:
df_player_matches_small = dd.read_csv('player_matches_small.csv', dtype=c_types, comment='\'')
dd.to_parquet(df_player_matches_small, 'player_matches_small_pk.parquet', engine='pyarrow')

In [ ]:
df_player_matches_small['account_id'].isna().sum().compute()

In [ ]:
len(df_player_matches_small)

In [ ]:
df_player_matches_small

In [40]:
# test pyspark json column reading
from pyspark.sql.functions import from_json, col
df = pd.read_csv('matches_small.csv', nrows=100000)
df.columns

Index(['match_id', 'match_seq_num', 'radiant_win', 'start_time', 'duration',
       'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant',
       'barracks_status_dire', 'cluster', 'first_blood_time', 'lobby_type',
       'human_players', 'leagueid', 'positive_votes', 'negative_votes',
       'game_mode', 'engine', 'picks_bans', 'parse_status', 'chat',
       'objectives', 'radiant_gold_adv', 'radiant_xp_adv', 'teamfights',
       'version', 'pgroup'],
      dtype='object')

In [43]:
df[['parse_status']].dropna().head(10)
# account_id is float with .0?
# gold is float with .0?
# gold per min is int
# xp per min is int

,parse_status
0,3.0
1,3.0
2,3.0
3,3.0
4,3.0
5,3.0
6,3.0
7,3.0
8,3.0
9,3.0
